In [1]:
!pip install -Uq langchain langchain-community langchain_unstructured chromadb langchain_google_genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.5 MB/s eta 0:00:00

In [1]:
import warnings
warnings.filterwarnings('ignore')

from langchain_unstructured  import UnstructuredLoader
from langchain_community.document_loaders import CSVLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder

In [2]:
from google.colab import userdata
langchain_api_key = userdata.get('LANGCHAIN_API_KEY')
google_api_key = userdata.get('GOOGLE_API_KEY')

In [3]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] = "RAG"
os.environ["GOOGLE_API_KEY"] = google_api_key

In [4]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
)

In [5]:
model = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-8b", convert_system_message_to_human=True)

In [6]:
print(model.invoke("hey").content)

Hey there!  How can I help you?


In [8]:
# md files
loader = UnstructuredLoader("/content/employee_handbook.md",metadata={"department": "general"})
loader = UnstructuredLoader("/content/engineering_master_doc.md",metadata={"department": "engineering"})
loader = UnstructuredLoader("/content/financial_summary.md",metadata={"department": "finance"})
loader = UnstructuredLoader("/content/market_report_q4_2024.md",metadata={"department": "marketing"})
loader = UnstructuredLoader("/content/marketing_report_2024.md",metadata={"department": "marketing"})
loader = UnstructuredLoader("/content/marketing_report_q1_2024.md",metadata={"department": "marketing"})
loader = UnstructuredLoader("/content/marketing_report_q2_2024.md",metadata={"department": "marketing"})
loader = UnstructuredLoader("/content/marketing_report_q3_2024.md",metadata={"department": "marketing"})
loader = UnstructuredLoader("/content/quarterly_financial_report.md",metadata={"department": "finance"})



In [9]:
documents = loader.load()

ImportError: unstructured package not found, please install it with `pip install unstructured`

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model = "gemini-1.5-flash-8b"
    chunk_size=500,
    chunk_overlap=50
    )
docs = text_splitter.split_documents(documents)

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=gemini_embeddings,
)
retriever = vectorstore.as_retriever()

In [ ]:
system_prompt = (
    '''
    You are an assistant for question‐answering tasks.

Each retrieved context piece has a metadata field `department` (e.g. “finance”, “hr”, “marketing”).
You should ONLY use context pieces whose `department` matches the user’s current role.
If the user’s role does not match any context, or if no matching context is available, respond with:
  “I’m sorry, but I don’t have access to that information.”

Use the following context to answer the user’s question.
If you don’t know the answer from the provided context, say “I don’t know.”

{context}
'''

)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answering_chain = create_stuff_documents_chain(model, chat_prompt)

In [ ]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [ ]:
print(rag_chain.invoke({"input": "I'm a manager in finance. give me a brief overview of marketing report in q3"}))